# Task 1

### Defining specifications for quantification of gene expression and guide count for different CRISPR+scRNA-seq protocols


### Relevance:
Kallisto bustools already have information about some assays, but we should be able to capture information from seqspec (https://github.com/IGVF/seqspec/tree/main/assays) and provide the read format and whitelist from different (10XV2,10XV3,10X5’) assays and create a structure that can extend for new future assays.


### Practical description: TODO
- [X] Create a new process (ex: “preprocessing_assay”). It should run before the alignment step.
- [ ] Create a new parameter in the perturb.config to receive the flag “params.ASSAY_SPECS”
- [ ] Create a table with the most common reads formats (10XV2, 10XV3, 5'PE... parse it from seqspec)  
- [ ] Create a custom script that can receive a “params.ASSAY_SPECS” ex: “10XV2”  and return the read parsing instructions (string format- see Kallisto bustools) and whitelist link (the script should download it)
- [ ] Make sure this module  will work only if params.ASSAY_SPECS is NOT 'custom' , otherwise it will try to capture informations from  (param.CHEMISTRY) and whitelist (params.WHITELIST) . If the  params.ASSAY_SPECS don't exist it interrupt the pipeline

obs: Setting assay is an artificial process (in the real script we need to modify the mappingscRNA  and mappingGuide   process)


### Kallisto read specification

"Additionally kallisto bus will accept a string specifying a new technology in the format of bc:umi:seq where each of bc,umi and seq are a triplet of integers separated by a comma, denoting the file index, start and stop of the sequence used. For example to specify the 10xV2 technology we would use 0,0,16:0,16,26:1,0,0. The first part bc is 0,0,16 indicating it is in the 0-th file (also known as the first file in plain english), the barcode starts at the 0-th bp and ends at the 16-th bp in the sequence (i.e. 16bp barcode), the UMI is similarly in the same file, right after the barcode in position 16-26 (a 10bp UMI), finally the sequence is in a separate file, starts at 0 and ends at 0 (in this case stopping at 0 means there is no limit, we use the entire sequence)."
https://pachterlab.github.io/kallisto/manual

---
## This Jupyter notebook has 2 scripts:  

*    jamboree_test.nf **  (instruction to execute the pipeline and defining process)  

*   check_aspects.py  (python script) 


 

Execute the last cell "!nextflow run .... " to debug

In [ ]:
%%writefile jamboree_test.nf

nextflow.enable.dsl=2

//or custom
params.ASSAY_SPECS = 'other'
//case custom this parameter will be used in the run
params.CHEMISTRY = '10XV2'
params.WHITELIST= '/home/jovyan/737K-august-2016.txt'


process preprocessing_assay{
    debug true
    input:
    val assay_specs
    output: 
    stdout
    script:
    """
    #script should return a csv
    #echo teste1,teste2
    chmod 700 /n/data1/bch/hemonc/bauer/lucassilva/gasperini_jamboree/check_assay_spects.py
    /n/data1/bch/hemonc/bauer/lucassilva/gasperini_jamboree/check_assay_spects.py assay_specs
    """
    
}



process setting_assay {
    debug true
    input:
    val (assay_value)
    val (chemistry)
    val (whitelist)
    val (new_params)
    script: 
    

    if(assay_value != 'custom')
    

    """
     echo 'using the assay_specs'
     echo kallisto using ${new_params[0]} and ${new_params[1]}

    """
    
    else
        
    """
     echo 'using custom chemistry and whitelist'
     echo kallisto using ${chemistry} and ${whitelist}


    """

}



workflow {

    out_process = preprocessing_assay(params.ASSAY_SPECS)
    new_values = out_process.view().splitCsv()
    setting_assay(params.ASSAY_SPECS, params.CHEMISTRY ,params.WHITELIST, new_values)
    params.WHITELIST
    
    
 
}

Writing jamboree_test.nf


In [ ]:
%%writefile check_assay_spects.py
#!/usr/bin/env python
import sys

assay_name = sys.argv[1]
if assay_name == 'custom':
    #case custom setting the return will not be used, but lets pring something to make easy the debug
    
    print ('custom,custom')
    exit()

#don't print anything inside this script
#check the assay_name
#download the specific whitelist
#format the chemistry given the assay_name (seqspec should help us) ex: "0,0,16:0,16,28:1,0,0" 



chemistry = 'python_chem' # return here the chemistry (check the kallisto format in the task description)

whitelist = 'python_white'  # return here the whitelist (first download it)


print (chemistry + ',' + whitelist )

Writing check_assay_spects.py


In [ ]:
!nextflow run jamboree_test.nf -w jamboree_test_task01

/bin/bash: nextflow: command not found
